![](1.png)
## 赛题名称
- Optiver - Trading at the Close
## 赛题链接
- https://www.kaggle.com/competitions/optiver-trading-at-the-close
# 赛题背景
证券交易所是快节奏、高风险的环境，每一秒都很重要。随着交易日接近尾声，强度不断升级，在关键的最后十分钟达到顶峰。这些时刻通常以波动加剧和价格快速波动为特征，在塑造当今全球经济叙事方面发挥着关键作用。

纳斯达克证券交易所的每个交易日都以纳斯达克收盘交叉拍卖结束。此过程确定了在交易所上市的证券的官方收盘价。这些收盘价是投资者、分析师和其他市场参与者评估个别证券和整个市场表现的关键指标。

在这个复杂的金融环境中，Optiver是全球领先的电子做市商。在技术创新的推动下，Optiver交易各种金融工具，如衍生品，现金股票，ETF，债券和外币，为全球主要交易所的数千种此类工具提供具有竞争力的双边价格。

在纳斯达克交易所交易时段的最后十分钟，像Optiver这样的做市商将传统的订单簿数据与拍卖簿数据合并。这种整合来自两个来源的信息的能力对于为所有市场参与者提供最优惠的价格至关重要。

# 赛题任务
在本次比赛中，您面临的挑战是开发一个模型，该模型能够使用订单簿和股票收盘价中的数据预测数百只纳斯达克上市股票的<font color='yellow'>收盘价走势</font>。拍卖信息可用于调整价格、评估供需动态以及识别交易机会。

# 评价指标
根据预测回报和观测目标之间的平均绝对误差 <font color='yellow'>（MAE）</font> 评估提交。公式由下式给出：
MAE = (1/n) * Σ(|y_i - y_hat_i|)
其中，n表示样本数量，y_i表示真实值，y_hat_i表示预测值。Σ表示求和符号，| |表示取绝对值。   **<font color='yellow'>越低越好</font>**

# 数据说明
[训练集/测试集].csv 拍卖数据。测试集将通过API提供。
- stock_id - 股票的唯一标识符。不会在所有时间段都出现所有的股票ID。<font color='yellow'>分析不同股票之间的行为和差异</font>

- date_id - 日期的唯一标识符。日期ID是顺序一致的，贯穿所有股票。<font color='yellow'>可用于将数据按日期进行分组和分析。</font>

- imbalance_size - 在当前参考价格下无法配对的金额(美元)。<font color='yellow'>不平衡大小，表示买卖不平衡的大小差异。可用于分析市场不平衡程度。</font>

- imbalance_buy_sell_flag - 反映拍卖不平衡方向的指标。<font color='yellow'>可用于分析市场趋势。</font>
    - 买方不平衡；1

    - 卖方不平衡；-1

    - 无不平衡；0

- reference_price - 配对股份最大化,不平衡最小化,且与买卖盘中点距离最小化的价格,依此顺序。可以视为受最佳买入和卖出价格限定的接近价格。<font color='yellow'>通常是开盘价，可用于分析价格相对于开盘价的变化。</font>

- matched_size - 在当前参考价格下可以配对的金额(美元)。<font color='yellow'>表示已匹配的订单的大小。可用于分析市场活动和交易量。</font>

- far_price - 仅基于拍卖利益可以配对最多股份的交叉价格。该计算不包括连续市场订单。

- near_price - 基于拍卖和连续市场订单可以配对最多股份的交叉价格。

- [bid/ask]_price - 非拍卖订单簿中最具竞争力的买入/卖出价格。<font color='yellow'>买单价格和买单大小，表示当前的买单信息，可用于分析市场深度和支撑水平。</font>

- [bid/ask]_size - 非拍卖订单簿最具竞争力的买入/卖出金额。<font color='yellow'>买单价格和买单大小，表示当前的买单信息，可用于分析市场深度和支撑水平。</font>

- wap - 非拍卖订单簿的加权平均价格。<font color='yellow'>是买卖价格的交易量加权平均值，可用于估计当前市场价格。</font>

- seconds_in_bucket - 从当日收盘拍卖开始经过的秒数，总是从0开始。<font color='yellow'>每个数据点在时间窗口内的秒数，可用于分析数据点的时间分布和行为。</font>

- target - 股票wap在未来60秒内的价格变动减去合成指数在未来60秒内的价格变动。

------------------------------

# Data EDA


In [1]:
import pandas as pd  # 数据处理
import numpy as np  # 科学运算
import matplotlib.pyplot as plt # 绘图
import seaborn as sns # 绘图
import warnings # 无视警告
warnings.filterwarnings('ignore')

# 配置文件
class CONFIG():
    train_path = '../train.csv'
    test_path = '../example_test_files/test.csv'
    
# 加载数据
train = pd.read_csv(CONFIG.train_path).drop(['row_id'], axis=1)
test = pd.read_csv(CONFIG.test_path)

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
train.describe().T

结论：
- stock_id：范围从0到199，近似平均值为99.29。不存在空值。
- date_id：范围从0到480，近似平均值为241.51。此外，不存在空值。
- seconds_in_bucket：以秒为单位的时间范围从0到540，平均约为270秒。
- imbalance_size：范围很广，从0到大约29.8亿。这可能表明存在异常值。
- imbalance_buy_sell_flag：这是一个二进制指示符，范围从-1到1。接近零的平均值表明买卖之间存在大致平衡。
- reference_price：平均而言，参考价格接近1，最低0.9353，最高1.0775。
- matched_size：与imbalance_size类似，它的范围很广，可能包含异常值。
- far_price和near_price：两者都接近1，但范围不同，这可能是进一步探索的有趣之处。
- bid_price和ask_price：两者都非常接近1，并且具有相似的方差。
- bid_size和ask_size：两者都有相似的均值和方差，表明订单簿在出价和要价方面可能相对平衡。
- wap（加权平均价格）：平均值接近1，类似于bid_Price和ask_Price。
- target：这是我们预测的目标。其平均值约为-0.0476，标准差较大，表明变异性较大。
- time_id：范围从0到26454，近似平均值为13310。
- Null值：列imbalance_size、reference_price、matched_size、far_price和near_price、bid_price、ask_price，wap和target具有一些需要处理的Null值。

In [ ]:
# 缺失值检查
train.isnull().sum()

imbalance_size、reference_price、matched_size、bid_price、ask_price和wap：这些列各缺少220个值。由于这些列中缺失值的数量相同，因此很可能是同一行中缺失了这些值。这将需要通过插补或删除这些行来解决。

far_price：此列有2894342个缺失值，明显高于其他列。需要特别注意处理这些缺失值，可能通过插补或使用不同的特征工程策略。

near_price：此列缺少2857180个值，也明显高于其他列，但少于far_price。与far_price类似的策略也可以应用于此。

target:缺少88个值。假设这是目标变量，则最有可能需要从训练集中删除缺少目标值的行。

In [ ]:
# 同类股票的平均WAP
# 计算每只股票的WAP平均值和标准差
grouped_stocks = train.groupby('stock_id')['wap'].agg(['mean', 'std']).reset_index()

# 根据平均WAP和标准差对股票进行排序
grouped_stocks.sort_values(by=['mean', 'std'], inplace=True)

# 创建由10只类似股票组成的小组
grouped_stocks['group'] = grouped_stocks.index // 10

# Plotting the mean WAP for the first 10 groups
plt.figure(figsize=(16, 8))
sns.boxplot(x='group', y='mean', data=grouped_stocks[grouped_stocks['group'] < 10])
plt.title('Mean WAP for Groups of 10 Similar Stocks')
plt.xlabel('Group')
plt.ylabel('Mean WAP')
plt.show()

In [ ]:
grouped_stocks.head(20).set_index('stock_id')

结论：
- WAP效果很好: 这些股票组的平均WAP（加权平均价格）徘徊在1！看起来物以类聚。
- 集体拥抱: 我们对股票进行了分组，所以每组中的股票就像失散多年的兄弟姐妹。它们具有相似的WAP平均值和标准偏差。
- 下一步是什么？: 知道哪些股票是好朋友可以很好的帮助我们在特征工程和构建强预测模型。
- 有离散: 如果一个群体中的一只股票开始表现得像独狼, 是时候举起红旗了. 可能是一个市场事件或其他值得挖掘的事情！



**将类似的股票分组就像组建一支支队伍. 它帮助我们更好地掌握市场动态，并可以完全提升我们的预测模型！**

In [ ]:
# 组中股票的WAP随时间变化
# 筛选这十组股票的id
stocks_in_first_10_groups = grouped_stocks[grouped_stocks['group'] < 10]['stock_id'].unique()

# 创建画布
fig, axes = plt.subplots(5, 2, figsize=(20, 30))

# 使轴阵列变平，便于索引
axes = axes.flatten()

# 循环浏览每组，绘制该组股票的WAP
for i in range(10):
    stocks_in_group = grouped_stocks[grouped_stocks['group'] == i]['stock_id'].unique()
    filtered_data = train[train['stock_id'].isin(stocks_in_group)]

    sns.lineplot(x='time_id', y='wap', hue='stock_id', data=filtered_data, ax=axes[i])
    axes[i].set_title(f'WAP Variation Over Time for Stocks in Group {i + 1}')
    axes[i].set_xlabel('Time')
    axes[i].set_ylabel('WAP')

plt.tight_layout()
plt.show()

常见模式: 如果一个组合中的多只股票显示出类似的WAP趋势， 代表对配对交易策略有用或多样化.
波动性: 观察WAP的起伏，发现过山车. 波动性更大的股票可能会提供更多快速获利的机会, 但别忘了戴帽子，太危险了！
分组验证: 情节反复检验我们的“物以类聚”是否真的“群策群力”. 如果没有，返回绘图板

In [ ]:
# 特征之间的相关性
correlation_matrix = train.corr()

plt.figure(figsize=(15, 10))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Feature Correlation Matrix')
plt.show()

- imbalance_size和matched_size：正相关是有道理的，因为这两个特征都与交易规模有关。正如相关性所反映的那样，更大的失衡确实可能导致匹配股票的规模更大。
- reference_price、bid_price、ask_price和wap：这些变量之间的高度相关性是合乎逻辑的，因为它们都与股价有关。具体而言，reference_price是最大股票数量匹配的价格，这自然类似于加权平均价格（wap）和最具竞争力的买卖价格。
- imbalance_buy_sell_flag和near_price：相关性表明，失衡的方向（买入或卖出）可能会影响near_prices，near_prince是最大化匹配股票数量的交叉价格之一。这是有道理的，因为买卖不平衡可能会影响大多数股票的交易价格。
- seconds_in_bucket和imbalance_size：负相关性很有趣，可能表明随着市场收盘（seconds_in_bucket越高），失衡可能会减少，可能是由于头寸的清算。
- target：它与其他变量没有很强的相关性，这表明预测wap的未来变化是复杂的，不能仅仅基于提供的其他变量。

In [ ]:
# 检查数据集中异常值的百分比
# 初始化列表以存储结果
outliers_info = []

# 计算每个数值列的异常值数量和百分比
for col in train.select_dtypes(include=['number']).columns:
    Q1 = train[col].quantile(0.25)
    Q3 = train[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = train[(train[col] < lower_bound) | (train[col] > upper_bound)]
    outliers_quantity = len(outliers)
    outliers_percentage = (outliers_quantity / len(train)) * 100
    outliers_info.append((col, outliers_quantity, round(outliers_percentage, 2)))

outliers_df = pd.DataFrame(outliers_info, columns=['Feature', '异常值数量', '异常值百分比'])

# 排序
outliers_df = outliers_df.sort_values(by='异常值百分比', ascending=False).reset_index(drop=True)
outliers_df = outliers_df[outliers_df['异常值百分比'] > 0]

outliers_df

- matched_size和imbalance_size：这些是与交易规模和当前参考价格下不匹配的金额相关的财务指标。此处的异常值可能表示异常市场事件，如大额买入/卖出或价格突然波动。

- bid_size、ask_size、far_price、near_price：这些也是财务指标，异常值可能是由于类似的市场事件。调查这些变量的异常值与其他变量（如imbalance_size和matched_size）之间是否存在任何相关性可能会很有趣。

- wap、bid_price、ask_price、reference_price和target：这些特征中的异常值也可能是罕见市场事件或波动的症状。

- seconds_in_bucket、imbalance_buy_sell_flag、stock_id、date_id、time_id：如前所述，这些是分类或基于时间的变量，没有异常值，这在给定其上下文的情况下是有意义的。

结论背后的逻辑：

- 背景分析：考虑到我们正在处理金融市场数据，不寻常的事件是意料之中的，可能对预测特别感兴趣。因此，直接去除异常值可能不是所有情况下的最佳策略。

- 建模：一些建模技术对异常值更具鲁棒性。根据您为比赛选择的算法，您可能会选择留下异常值或以某种方式处理它们。

- 相关性：由于这些变量与市场交易的特定方面有关，了解不同变量中异常值之间的相关性可以深入了解异常市场事件。

- 特征工程：您可能希望创建新的变量，以对模型更有用的方式捕获异常值中包含的信息。

<font color='yellow'>验证：任何处理异常值的策略都应该经过仔细验证。</font>

In [ ]:
# 研究不同变量中异常值之间的相关性
# 识别异常
def identify_outliers(df, col):
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)]
    return outliers

# 识别“matched_size”和“imbalance_size”的异常值
outliers_matched_size = identify_outliers(train, 'matched_size')
outliers_imbalance_size = identify_outliers(train, 'imbalance_size')

# 检查“matched_size”和“imbalance_size”之间是否存在重复行（异常值）
common_outliers = pd.merge(outliers_matched_size, outliers_imbalance_size, how='inner')

# 计算相关性
correlation_matrix = common_outliers.corr()

correlation_matrix[['imbalance_size', 'matched_size']].T

考虑到特征描述，结论变得更加相关：
- imbalance_size和matched_size之间的适度相关性表明，以当前参考价格（以美元计）不匹配的金额和以当前参考价（以美元为单位）可以匹配的金额在某种程度上是相关的。这可能有助于预测或了解市场行为。
- imbalance_size和seconds_in_bucket之间的负相关性可能表明，自当天收盘拍卖开始以来，随着时间的推移，不匹配的数量可能会减少。这可能是一个值得探索的市场现象。
- matched_size和seconds_in_bucket之间的正相关性可能表明，随着收盘拍卖的临近，匹配的交易越来越多。
- 此外，这些变量与目标之间缺乏显著相关性（股票WAP未来60秒的波动），这表明它们可能不是价格波动的良好直接预测因素，但在更复杂的模型中或与其他变量组合时可能有用。

# Baseline

In [2]:
# 优化内存的函数
def reduce_mem_usage(df):
    start_mem = df.memory_usage().sum() / 1024**2
    print(f'Memory usage of dataframe is {start_mem:.2f} MB')

    for col in df.columns:
        col_type = df[col].dtype

        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float32)
    end_mem = df.memory_usage().sum() / 1024**2
    print(f'Memory usage after optimization is: {end_mem:.2f} MB')
    decrease = 100 * (start_mem - end_mem) / start_mem
    print(f'Decreased by {decrease:.2f}%')

    return df

In [3]:
from itertools import combinations
import gc
# 每个股票的bid_size和ask_size的中位数之和
median_vol = train.groupby('stock_id')['bid_size'].median() + train.groupby('stock_id')['ask_size'].median()

def feat_eng(df):

    cols = [c for c in df.columns if c not in ['row_id', 'time_id']]
    df = df[cols]
    # 将买入不平衡和卖出不平衡的标志转化为二进制标志
    df['imbalance_buy_flag'] = np.where(df['imbalance_buy_sell_flag']==1, 1, 0)
    df['imbalance_sell_flag'] = np.where(df['imbalance_buy_sell_flag']==-1, 1, 0)
    
    # 订单薄中的买单大小和卖单大小之和
    df['bid_plus_ask_sizes'] = df['bid_size'] + train['ask_size']
    # 每个股票的bid_size和ask_size的中位数之和
    df['median_vol'] = df['stock_id'].map(median_vol.to_dict())
    # 前订单簿的买卖总量是否高于该股票的median_vol，如果是，则为1，否则为0。
    df['high_volume'] = np.where(df['bid_plus_ask_sizes'] > df['median_vol'], 1, 0)
    # 不平衡大小（imbalance_size）与匹配大小（matched_size）之间的比率。
    df['imbalance_ratio'] = df['imbalance_size'] / df['matched_size']
    # 表示买入-卖出不平衡和不平衡大小-匹配大小之间的比率。
    df['imb_s1'] = df.eval('(bid_size-ask_size)/(bid_size+ask_size)')
    df['imb_s2'] = df.eval('(imbalance_size-matched_size)/(matched_size+imbalance_size)')
    # 表示买单和卖单的加权大小，将价格与数量相乘。
    df['ask_x_size'] = df.eval('ask_size*ask_price')
    df['bid_x_size'] = df.eval('bid_size*bid_price')
    # 表示卖单和买单的加权大小之间的差异。
    df['ask_minus_bid'] = df['ask_x_size'] - df['bid_x_size']
    # 表示买单大小和价格与卖单大小和价格之间的比率。
    df["bid_size_over_ask_size"] = df["bid_size"].div(df["ask_size"])
    df["bid_price_over_ask_price"] = df["bid_price"].div(df["ask_price"])

    prices = ['reference_price','far_price', 'near_price', 'ask_price', 'bid_price', 'wap']
    # 各种价格特征的差异、乘积和不平衡比率
    for c in combinations(prices, 2):

        df[f'{c[0]}_minus_{c[1]}'] = (df[f'{c[0]}'] - df[f'{c[1]}']).astype(np.float32)
        df[f'{c[0]}_times_{c[1]}'] = (df[f'{c[0]}'] * df[f'{c[1]}']).astype(np.float32)
        df[f'{c[0]}_{c[1]}_imb'] = df.eval(f'({c[0]}-{c[1]})/({c[0]}+{c[1]})')
    # 组合三个价格特征的不平衡比率
    for c in combinations(prices, 3):
        max_ = df[list(c)].max(axis=1)
        min_ = df[list(c)].min(axis=1)
        mid_ = df[list(c)].sum(axis=1)-min_-max_

        df[f'{c[0]}_{c[1]}_{c[2]}_imb2'] = (max_-mid_)/(mid_-min_)
    df.drop(columns=['date_id'], inplace=True)


    df=reduce_mem_usage(df)
    gc.collect()
    return df

In [ ]:
train.columns

- 'imbalance_buy_flag'和 'imbalance_sell_flag'：买入和卖出不平衡标志，用于识别买卖不平衡的方向。

- 'bid_plus_ask_sizes'：买单和卖单大小之和，可用于分析市场深度。

- 'median_vol'：每个股票的买卖大小中位数，用于分析市场活动。

- 'high_volume'：指示当前市场活动是否高于中位数，用于分析高交易量时的行为。

- 'imbalance_ratio'：不平衡大小与匹配大小之间的比率，用于分析不平衡情况。

- 'imb_s1'和 'imb_s2'：与买卖不平衡相关的衍生特征，用于分析买卖不平衡的趋势。

- 'ask_x_size'和 'bid_x_size'：买单和卖单的交易量加权价格，可用于估计市场价格。

- 'ask_minus_bid'：卖单和买单的加权价格之间的差异，用于分析价格差异。

- 'bid_size_over_ask_size'和 'bid_price_over_ask_price'：买单大小和价格与卖单大小和价格之间的比率，可用于分析价格和交易量之间的关系。

- 其他价格特征：包括价格差异、价格变化速度、价格均值和标准差等，用于分析价格的变化和波动性。


In [4]:
train.dropna(subset=['target'], inplace=True)

In [5]:
y = train['target']
X = feat_eng(train.drop(columns='target'))

Memory usage of dataframe is 3017.13 MB
Memory usage after optimization is: 1218.84 MB
Decreased by 59.60%


In [6]:
from timeit import default_timer as timer
from sklearn.model_selection import train_test_split, TimeSeriesSplit
from sklearn.metrics import mean_absolute_error
import lightgbm as lgb
# 交叉验证
def cross_validate(model, X, y, cv):
    scores = np.zeros(cv.n_splits)
    for i, (train_index, test_index) in enumerate(cv.split(X)):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, shuffle=False, test_size=0.1)
        start = timer()
        model.fit(X_train, y_train, eval_set=[(X_val, y_val)], callbacks=[lgb.early_stopping(50, verbose=False)])
        end = timer()
        y_pred = model.predict(X_test)
        scores[i] = mean_absolute_error(y_pred, y_test)
    return scores

In [7]:
# 评估指标
def evaluate_simple(model, X, y, cv):
    X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=False, test_size=0.2)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, shuffle=False, test_size=0.1)
    start = timer()
    model.fit(X_train, y_train, eval_set=[(X_val, y_val)], callbacks=[lgb.early_stopping(50, verbose=False)])
    end = timer()
    y_pred = model.predict(X_test)
    score = mean_absolute_error(y_pred, y_test)
    return score

In [8]:
import optuna
import json

# 超参数优化的函数
## objective:要优化的函数 n_tri：试验的次数 n_jobs:并行册数
def run_optimization(objective, n_trials=100, n_jobs=1):
    optuna.logging.set_verbosity(optuna.logging.WARNING)
    study = optuna.create_study(direction="minimize")
    study.optimize(objective, n_trials=n_trials, n_jobs=n_jobs, show_progress_bar=True)
    with open("best_params.json", "w") as f:
        json.dump(study.best_params, f)
    return study

In [9]:
from lightgbm import LGBMRegressor

# 为Optuna超参数优化提供一个目标函数
def get_objective_function(evaluation="simple", cv=None, logging_level="info"):
    """Returns the objective function for optuna."""
    if evaluation == "simple":
        eval_function = evaluate_simple
    else:
        eval_function = cross_validate

    def optimize_lgbm(trial):
        """Optimizes a LGBMRegressor with cross-validation."""
        # num_leaves should be smaller than 2^{max_depth}
        max_depth = trial.suggest_int("max_depth", 6, 9)
        num_leaves = trial.suggest_int("num_leaves", 32, int((2**max_depth) * 0.90))

        param_space = {
            "boosting":'gbdt',
            "objective": trial.suggest_categorical("objective", ["mae"]),
            "random_state": trial.suggest_categorical("random_state", [42]),
            "n_estimators": trial.suggest_categorical("n_estimators", [600]),
            "reg_alpha": trial.suggest_float("reg_alpha", 1e-3, 1.0, log=True),
            "reg_lambda": trial.suggest_float("reg_lambda", 1e-3, 1.0, log=True),
            "learning_rate": trial.suggest_float("learning_rate", 1e-2, 2e-1, log=True),
            "num_leaves": num_leaves,
            "max_depth": max_depth
        }
        model = LGBMRegressor(**param_space)
        scores = eval_function(model, X, y, cv=cv)
        return scores.mean()
    return optimize_lgbm

In [ ]:
m = lgb.LGBMRegressor(objective='mae', n_estimators=600, random_state=51)
m.fit(X, y)

In [ ]:
import plotly.express as px

feat_imp = pd.Series(m.feature_importances_, index=X.columns).sort_values(ascending=False)
print('Columns with poor contribution', feat_imp[feat_imp<10].index)
fig = px.bar(x=feat_imp, y=feat_imp.index, orientation='h')
fig.show()

In [ ]:
test=feat_eng(test)

# 开始优化

可以设置以下参数进行模型优化：

- run_lgbm_optimization：是否运行优化或使用已计算的优化。
- n_trials：我们想要采样多少次试验。
- logging_level：配置评估函数内的日志记录级别（使用“info”或“success”）
- 评估：使用“simple”进行简单的训练-测试分割，或使用“cross_validate”使用 TimeSeriesSplit 进行交叉验证。
- cv：分割对象
- 警告：evaluation='cross_validate'需要很长时间！

In [ ]:
gc.collect()

In [ ]:
run_lgbm_optimization = True
n_trials = 30
logging_level = "success"
evaluation = "simple"
cv = TimeSeriesSplit(n_splits=3)


from IPython.display import clear_output
from optuna.visualization import (
    plot_optimization_history,
    plot_param_importances,
    plot_parallel_coordinate,
)

if run_lgbm_optimization:
    clear_output(wait=True) 
    objective = get_objective_function(evaluation=evaluation, cv=cv)
    study = run_optimization(objective, n_trials=n_trials, n_jobs=1)
    lgbm_best_params = study.best_params

    plot_optimization_history(study).show()
    if n_trials > 1:
        plot_param_importances(study).show()
        plot_parallel_coordinate(study).show()
else:
    lgbm_best_params = {}

# 模型推理

In [ ]:
# 如果你优化了，请重新训练并保存
# model = LGBMRegressor(**lgbm_best_params)
# model.fit(X,y)

m.predict(test)

# 提交

In [ ]:
# kaggle
# import optiver2023
# env = optiver2023.make_env()
# iter_test = env.iter_test()

# local
from public_timeseries_testing_util import MockApi
env=MockApi()
iter_test = env.iter_test()

In [ ]:
test.head()

In [ ]:
def zero_sum(prices, volumes):

    std_error = np.sqrt(volumes)
    step = np.sum(prices)/np.sum(std_error)
    out = prices-std_error*step

    return out

In [ ]:
counter = 0
for (test, revealed_targets, sample_prediction) in iter_test:

    feat = feat_eng(test)
    sample_prediction['target'] = m.predict(feat)

    sample_prediction['target'] = zero_sum(sample_prediction['target'], test.loc[:,'bid_size'] + test.loc[:,'ask_size'])

    env.predict(sample_prediction)

    counter += 1

In [ ]:
# 模型保存
import joblib
joblib.dump(m,'./baseline1_model')